In [207]:
import os
import numpy as np

## Positive Expected Value Betting Routine

Project idea / design:

Want: A general routine for finding the EV and Kelly Criterion bet amount for a given boosted bet. 

Input: 
    1. given odds of boosted event. 
    2. given odds of each subevent
    3. given odds of each complement subevent
    
Alternate input (WIP): if we don't have the given odds of each subevent and complement subevent, we will develop an ML model to predict the VIG for an event, given event odds and event type (moneyline, spread, etc.)
    
F1: Take inputs (2,3) and calculate the vig of each subevent.  
F2: Use vig from F1 to find best estimate of real probability for each subevent and complement subevent.   
F3: Calculate probability of the boosted (main) event.   
F4: Use Kelly criterion to determine betting amount. 

Output: Wager amount for a given bankroll. Given the nature of Boost Bets, this will have a maximum of $50$ and a minimum of $0$, with $0 being the output for a negative EV bet. 



For example: We have three subevents, showing given odds that three different NBA players score 30+ points in a game on a given night.   
P1: -174	+130  
P2: -320	+225  
P3: -190	+152  

In [213]:
num_sub_events = int(float(input("How many subevents do you have? ")))

How many subevents do you have? 3


In [214]:
print(f"The boosted event has a total of {num_sub_events} subevents.")

The boosted event has a total of 3 subevents.


In [215]:
arr = input("Input tuples of subevent odds and complement subevent odds. ") 
#for example, if a given subevent has odds +130, complement odds -140, input: 130 -140
# if the next subevent has odds +300, complement odds -400, input 300 -400 in the same line, directly after the first two numbers
l = list(map(int,arr.split(' '))) 

Input tuples of subevent odds and complement subevent odds. -110 -110 -110 -110 -110 -110


In [216]:
subevent_array = np.array(l).reshape(num_sub_events, 2)
vig = np.empty((num_sub_events, 1))
subevent_complement_prob = np.empty((num_sub_events, 2))

In [217]:
subevent_array

array([[-110, -110],
       [-110, -110],
       [-110, -110]])

In [218]:
def calculate_vig(subevent_array): 
    for i in range(0, num_sub_events):
        if subevent_array[i][0] > 0: # 1st column, i.e. subevent odds
            subevent_complement_prob[i][0] = 100/(subevent_array[i][0]+100)
        else: 
            subevent_complement_prob[i][0] = abs(subevent_array[i][0]) / (abs(subevent_array[i][0]) + 100)
        if subevent_array[i][1] > 0: #2nd column, i.e. complement subevent odds
            subevent_complement_prob[i][1] = 100/(subevent_array[i][1]+100)
        else: 
            subevent_complement_prob[i][1] = abs(subevent_array[i][1]) / (abs(subevent_array[i][1]) + 100)
        vig[i][0] =  subevent_complement_prob[i][0]+ subevent_complement_prob[i][1]-1
    
    return vig, subevent_complement_prob
    
    

In [219]:
vig, subevent_complement_prob = calculate_vig(subevent_array)

In [220]:
def calculate_actual_probs(subevent_complement_prob, vig):
    actual_probs = np.empty((num_sub_events, 2))
    for i in range(num_sub_events):
        actual_probs[i] = np.divide(subevent_complement_prob[i],1+vig[i])
    return actual_probs

In [221]:
actual_probs = calculate_actual_probs(subevent_complement_prob, vig)

If you want to do a simple parlay of the subevents, follow the below cell.

In [222]:
def intersect_subevent_prob(actual_probs):
    """
   Input the matrix with shape (number of sub events, 2) that contains true probabilities for each event. 
    Assumes the subevents are independent. 
    Outputs the probability of all subevents occurring simultaneously. 
    """
    intersection_prob = np.empty((1,2))
    intersection_prob = np.prod(actual_probs, axis = 0)
    return intersection_prob[0]
    

In [237]:
true_probability = intersect_subevent_prob(actual_probs)

In [238]:
print(f"The true probability of all subevents occurring simultaneously is {np.round(true_probability*100,1)}%. ")

The true probability of all subevents occurring simultaneously is 12.5%. 


In [239]:
def calculate_fair_odds(true_probability):
    """
    Input: true_probability should be a float between 0 and 1.
    Output: Fair odds on the usual betting scale (100+ representing unlikely events, -100- representing likely events)
    """
    if true_probability <= .5:
        true_odds = 100/true_probability -100
    elif true_probability > .5 and true_probability <= 1:
        true_odds = -100/(1-true_probability)+100
    else:
        "Invalid input probability"
    return true_odds

It might be the case that the boost you're considering is not comprised of *only* intersections (i.e. only a parlay).  
There may also be the union of several events, or some combination of unions and intersections.  
A *union* is the logical condition where, given some number of events, when *at least* one event is true, the entire union is true.  For example, the statemen "player 1 scores 30+ points, *or* player 1 scores less than 30 points" is always true, since one or the other events must always occur. 

Use the below cell to calculate the union of two subevents. 

In [240]:
actual_probs

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

In [241]:
def union_subevent_prob(actual_probs, row_1, row_2):
    """
    Input a matrix of probabilities to take the union of. At most 2 probabilities. Uses the inclusion exclusion formula.
    Input the two rows (index starts at 0!) whose probabilities you would like to union. 
    Output probability of inclusive or. 
    """
    union_prob = actual_probs[row_1]+actual_probs[row_2]-(actual_probs[row_1]*actual_probs[row_2])
    
    return union_prob
    
    
  

In [242]:
union_prob = union_subevent_prob(actual_probs, 0,1)

In [243]:
print(f"The probability of P1 scoring 30+ points, or P2 scoring 30+ points, or both, is {np.round(union_prob[0]*100,1)}%. ")
print(f"The probability of P1 NOT scoring 30+ points, or P2 NOT scoring 30+ points, or of neither player scoring 30+ points, is {np.round(union_prob[1]*100,1)}%. ")

The probability of P1 scoring 30+ points, or P2 scoring 30+ points, or both, is 75.0%. 
The probability of P1 NOT scoring 30+ points, or P2 NOT scoring 30+ points, or of neither player scoring 30+ points, is 75.0%. 


Finally, we might wonder how much we should bet on a given event. We should consider two things:  
 1. What is the expected value of the bet? In other words, for every dollar we bet, what do we expect to return? If this isn't positive,  *don't bet*. 
 2. Assuming we have a limited bankroll, how much of it should we place on this bet? We'll use a formula called the *Kelly Criterion* to determine this. 

In [244]:
def kelly_bet_percentage(true_probability, odds_offered, bankroll_amt, boost_bet = True):
    """
    Input the true probability (estimated) of the main event happening. Input the odds offered you by the sportsbook. 
    Input the dollar value of your betting bankroll.
    Output the proportion of the bankroll you should risk (maximum $50 if this is a boosted bet). If 0 is the output, this means don't bet. 
    Output the dollar amount you should risk. 
    
    NB: kelly formula is f = p - (1-p)/b, where p is true probability, b is the proportion of each dollar bet that you would gain with a win. 
    e.g. if the odds offered are +200, you will win 2.0 dollars for every dollar bet. If the odds offered are -400, you win .25 for every dollar bet. 
    """
    
    if odds_offered > 0: 
        b_value = odds_offered/100
    else:
        b_value = 100/abs(odds_offered)
    
    kelly_proportion = true_probability - (1-true_probability)/b_value
    if kelly_proportion >=0 and boost_bet ==True:
        dollar_value_to_bet = min(kelly_proportion*bankroll_amt, 50)
    elif kelly_proportion >= 0 and boost_bet == False:
        dollar_value_to_bet = kelly_proportion*bankroll_amt
    else: 
        print("Don't bet on this!")
        dollar_value_to_bet =0 
    
    return kelly_proportion, dollar_value_to_bet
        

In [249]:
br_amt = 10**3
boosted = True
if boosted:
    place = ""
else: 
    place = "not"
prop, dol = kelly_bet_percentage(true_probability, 700, br_amt, boosted)

In [250]:
print(f"Our calculations say that, for the example event with given +330 odds, you should bet ${np.round(dol,2):,}, given you have a bankroll amount of ${br_amt:,}. Note: this assumes that the boost maximum of $50 is{place} in effect. ")

Our calculations say that, for the example event with given +330 odds, you should bet $0.0, given you have a bankroll amount of $1,000. Note: this assumes that the boost maximum of $50 is in effect. 


Finally, let's compute the expected value (EV) for this bet. 

In [251]:
def compute_ev(true_probability, given_odds):
    fair_odds = calculate_fair_odds(true_probability)
    if fair_odds >0 and given_odds > 0:
        ev = true_probability* (given_odds - fair_odds)
    elif fair_odds > 0 and given_odds < 0:
        ev = true_probability*(10000/abs(given_odds) - fair_odds)
        print("Don't bet on this!")
    elif fair_odds < 0 and given_odds < 0:
        ev = true_probability* (given_odds - fair_odds)
    elif fair_odds <0 and given_odds >0:
        ev = true_probability*(given_odds - 10000/abs(fair_odds))
    else: 
        print("Don't bet on this!")
    return np.round(ev,1)

In [255]:
ev = compute_ev(.125, 1000)
print(f"So, expect {ev}% return on each dollar bet on this offer. ")

So, expect 37.5% return on each dollar bet on this offer. 
